In [ ]:
# ===============================================================
# Colab Bulk Inference — LLaMA‑3.1‑8B + LoRA (hard‑merge)
# Input : CSV "ID;name;text" (UTF‑8‑SIG, semicolon)
# Output: CSV "ID;name;text;pred;pred_short;raw" (UTF‑8‑SIG, semicolon)
# Resume: Wenn Ausgabe existiert, überspringe bereits verarbeitete IDs
# Batching: Slice-basierte Chunks mit .iloc; nach jeder Batch gespeichert
# ===============================================================

# !pip -q install "transformers>=4.41.0" "accelerate>=0.30.0" sentencepiece peft huggingface_hub tqdm

import os, re, time, hashlib, warnings, csv
os.environ["TRANSFORMERS_VERBOSITY"] = "error"

from pathlib import Path
import pandas as pd
import torch
from tqdm import tqdm
from huggingface_hub import login
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
transformers.logging.set_verbosity_error()
warnings.filterwarnings("ignore", message=".*generation flags are not valid.*")

# Konfiguration
INPUT_CSV   = "/content/augmentation_prepped_for_inference.csv"
OUTPUT_CSV  = "/content/augmentation_prepped_for_inference_preds_llama.csv"
BATCH_SIZE  = 2500  # ← Slice-Größe; nach jeder Batch gespeichert

HF_REPO_ID  = "YangZexi/llama-3.1-8B-Instruct-stance-lora-v2"
BASE_MODEL  = "meta-llama/Meta-Llama-3.1-8B-Instruct"

MAX_LEN_IN  = 512
MAX_NEW_TOK = 3
DO_SAMPLE   = False
TEMPERATURE = 0.0  # nur verwendet, wenn DO_SAMPLE=True

# Optionaler Login
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get("HF_TOKEN")
except Exception:
    HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    login(token=HF_TOKEN)

# Helferfunktionen
_CLEAN_RE = re.compile(r'[\"\'\.\,\:\;\!\?\-\—\–\(\)\[\]\{\}]')
def parse_short_to_label(raw_output: str):
    if not isinstance(raw_output, str):
        return "Unklar", ""
    t = _CLEAN_RE.sub(" ", raw_output.strip().lower())
    t = re.sub(r"\s+", " ", t).strip()
    if not t:
        return "Unklar", ""
    first = t.split()[0]
    if first.startswith("zu"):  return "Zustimmung", "Zu"
    if first.startswith("ab"):  return "Ablehnung", "Ab"
    if first.startswith("ne"):  return "Neutral", "Ne"
    if "zu" in t: return "Zustimmung", "Zu"
    if "ab" in t: return "Ablehnung", "Ab"
    if "ne" in t: return "Neutral", "Ne"
    return "Unklar", ""

def pick_attn_impl():
    try:
        _ = torch.backends.cuda.sdp_kernel
        return "sdpa"
    except Exception:
        return "eager"

def tensor_md5(t):
    try:
        return hashlib.md5(t.detach().float().cpu().numpy().tobytes()).hexdigest()
    except Exception:
        return "NA"

def build_messages(name: str, text: str):
    system_msg = {
        "role": "system",
        "content": (
            "Du bist ein Stance-Klassifizierer für politische Tweets. "
            "Kategorisiere die Haltung als genau eine der drei Klassen: Zustimmung, Ablehnung oder Neutral."
        ),
    }
    user_prompt = (
        f"### Aufgabe\n"
        f"Bewerte die Haltung des folgenden Tweets gegenüber \"{name}\".\n\n"
        f"Tweet: {text}\n\n"
        "### Antwortmöglichkeiten:\n"
        "• Zustimmung: explizit/implizit positiv/unterstützend.\n"
        "• Ablehnung: explizit/implizit negativ/kritisch.\n"
        "• Neutral: sachlich/ambivalent/keine erkennbare Haltung.\n"
        "### Ausgabeformat (Kurzform):\n"
        "Gib **genau eines** der folgenden Kürzel zurück (ohne Anführungszeichen, ohne Punkt):\n"
        "Zu\nAb\nNe"
    )
    return [system_msg, {"role": "user", "content": user_prompt}]

# Modell laden & LoRA hard‑mergen
peft_cfg = PeftConfig.from_pretrained(HF_REPO_ID)
if peft_cfg.base_model_name_or_path and peft_cfg.base_model_name_or_path != BASE_MODEL:
    print(f"Adapter base={peft_cfg.base_model_name_or_path} -> überschreibe BASE_MODEL.")
    BASE_MODEL = peft_cfg.base_model_name_or_path

dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=dtype,
    trust_remote_code=True,
    attn_implementation=pick_attn_impl(),
)

probe_name = next((k for k in base.state_dict().keys() if k.endswith(".weight")), None)
pre_hash = tensor_md5(base.state_dict()[probe_name]) if probe_name else "NA"

model = PeftModel.from_pretrained(base, HF_REPO_ID)
model = model.merge_and_unload(safe_merge=True)
model.eval()

post_hash = tensor_md5(model.state_dict()[probe_name]) if probe_name else "NA"
print("Merge OK" if pre_hash != post_hash else "Merge-Probe unverändert — Adapter prüfen.")

# Daten laden & per ID fortsetzen
in_path  = Path(INPUT_CSV)
out_path = Path(OUTPUT_CSV)

df = pd.read_csv(in_path, sep=";", encoding="utf-8-sig", dtype=str)
df = df[["ID", "name", "text"]].dropna(subset=["ID", "text"]).copy()

processed_ids = set()
if out_path.exists():
    print(f"Fortsetzen: bestehende Ausgabe gefunden -> {out_path}")
    try:
        done = pd.read_csv(out_path, sep=";", encoding="utf-8-sig", dtype=str, usecols=["ID"])
        processed_ids = set(done["ID"].astype(str).tolist())
    except Exception as e:
        print(f"Konnte bestehende Ausgabe nicht lesen: {e}")

remain = df[~df["ID"].astype(str).isin(processed_ids)].reset_index(drop=True)
total  = len(df)
todo   = len(remain)
print(f"Gesamtzeilen: {total} | Bereits erledigt: {total - todo} | Zu erledigen: {todo}")

write_header = not out_path.exists()

# Einzelvorhersage
def predict_one(name: str, text: str):
    messages = build_messages(name, text)
    chat_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    enc = tokenizer(chat_text, return_tensors="pt", truncation=True, max_length=MAX_LEN_IN).to(model.device)

    gen_kwargs = dict(max_new_tokens=MAX_NEW_TOK, do_sample=DO_SAMPLE, pad_token_id=tokenizer.eos_token_id)
    if DO_SAMPLE:
        gen_kwargs["temperature"] = TEMPERATURE

    with torch.no_grad():
        out_ids = model.generate(**enc, **gen_kwargs)

    prompt_len = enc["input_ids"].shape[1]
    raw = tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip()
    label, short = parse_short_to_label(raw)
    return label, short, raw

# Slice-basierte Batchverarbeitung mit .iloc
start_ts = time.time()
for start in range(0, todo, BATCH_SIZE):
    batch = remain.iloc[start:start + BATCH_SIZE].copy()
    if batch.empty:
        continue

    rows = []
    pbar = tqdm(total=len(batch), desc=f"Batch {start//BATCH_SIZE + 1}", dynamic_ncols=True)
    for _, r in batch.iterrows():
        try:
            name = (r["name"] or "").strip()
            text = (r["text"] or "").strip()
            pred, pred_short, raw = predict_one(name, text)
        except Exception as e:
            pred, pred_short, raw = "Unklar", "", f"ERROR: {e}"

        rows.append({
            "ID": str(r["ID"]),
            "name": name,
            "text": text,
            "pred": pred,
            "pred_short": pred_short,
            "raw": raw
        })
        pbar.update(1)
    pbar.close()

    # Speichern dieser Batch sofort (Anhängemodus nach dem ersten Schreiben)
    pd.DataFrame(rows).to_csv(
        out_path,
        sep=";",
        index=False,
        encoding="utf-8-sig",
        mode=("w" if write_header else "a"),
        header=write_header,
        quoting=csv.QUOTE_MINIMAL
    )
    write_header = False

    elapsed = time.time() - start_ts
    done_now = min(start + BATCH_SIZE, todo)
    print(f"Batch {start//BATCH_SIZE + 1} gespeichert -> {out_path.name} | Erledigt: {done_now}/{todo} | {elapsed/60:.1f} min")

print(f"Abgeschlossen. Ausgabe -> {out_path}")

adapter_config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

💾 Saved batch 30 → augmentation_prepped_for_inference_preds_llama.csv | Done: 75000/398704 | 100.2 min


🔍 Batch 31: 100%|██████████| 2500/2500 [03:17<00:00, 12.68it/s]


💾 Saved batch 31 → augmentation_prepped_for_inference_preds_llama.csv | Done: 77500/398704 | 103.5 min


🔍 Batch 32:  60%|██████    | 1506/2500 [02:00<01:18, 12.61it/s]

KeyboardInterrupt: 